In [0]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [0]:
import cv2
import numpy as np
import os
import fnmatch
import re
import math
import torch
import glob
import pandas as pd
import matplotlib.pyplot as plt
from numpy import genfromtxt


def processvideo(videopath,trailer_names,channels,timeDepth,xSize,ySize):
   
    inputs = []
    filelist = []
    for filename in trailer_names:
        filelist.append(videopath+str(filename)+".mp4")
    print("Total Videos: ", len(filelist))
    frametensor = torch.FloatTensor(len(filelist),channels, timeDepth, xSize, ySize)
    try:
        for num, videofile in enumerate(filelist):
            cap = cv2.VideoCapture(videofile)
            frames = []
            fps    = cap.get(cv2.CAP_PROP_FPS)
            length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) #Length of the video in seconds
            flag = True
            cnt = 0
            hist_old, hist_new = None, None
            for cnt in range(length):
                ret,frame = cap.read()
                if cnt % 4 == 0:
                    if ret:
                        frame = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY) #Convert BGR to GRAY
                        frame = cv2.resize(frame,(xSize,ySize))
                        hist_new = cv2.calcHist([frame], [0], None, [256], [0,256])
                        if np.sum(hist_new[:1]) > np.sum(hist_new) * 0.5:
                            continue
                        hist_new = hist_new[1:256]
                        if flag:
                            hist_old = hist_new
                            flag = False

                        cor_hist = cv2.compareHist(hist_old, hist_new, cv2.HISTCMP_CORREL)
                        if abs(cor_hist) < 0.8:
                            frames.append(frame)
                            hist_old = hist_new
          
            nframes = len(frames)
            select_fps  = nframes/(channels*timeDepth)
            split_values  = [math.ceil(select_fps), math.floor(select_fps)]
            split_ind = round(select_fps - split_values[1])
            if split_ind == 0:
                split_ind = 1
            else:
                split_ind = 0
            split = split_values[split_ind]
            cnt = 0
            cnt_flag = True
            for ch in range(channels):
                for td in range(timeDepth):
                    if cnt >= split*channels*timeDepth*abs(select_fps-split_values[abs(split_ind-1)]) and cnt_flag:
                        cnt_flag = False
                        cnt_chng = ch*timeDepth+td
                        split = split_values[abs(split_ind-1)]
                        if not split % 2 is 0:
                            cnt -= split
                    if not split is 0 and cnt<nframes:
                        frame = torch.from_numpy(frames[cnt])
                        frametensor[num, ch, td, :, :] = frame
                    else:
                        frame = torch.from_numpy(frames[nframes-cnt-1])
                        frametensor[num, ch, td, :, :] = frame
                    cnt += split
            print(str(num) + "\t" + videofile + "\t[" + str(timeDepth*channels) + "/" + str(nframes) + "]\t" + str(cnt_chng) + "\t" + str(96-cnt_chng))
        return frametensor
    except:
        print("Exception", videofile)

timeDepth = 16
channels = 6
xSize, ySize = 58, 58
trailer_names = "gdrive/My Drive/Project - Box Office Prediction/Dataset/Metadata/final_data.csv"
trailer_names = pd.read_csv(trailer_names)['videoname'][train]
videopath = "gdrive/My Drive/Project - Box Office Prediction/Videos/"
frametensor = processvideo(videopath, trailer_names, channels,timeDepth,xSize, ySize)
torch.save(frametensor,'gdrive/My Drive/LTCTensor/TrailerTensor_1489_augmented_train.pt')

Total Videos:  1191
0	gdrive/My Drive/Project - Box Office Prediction/Videos/19.mp4	[96/327]	57	39
1	gdrive/My Drive/Project - Box Office Prediction/Videos/670.mp4	[96/176]	80	16
2	gdrive/My Drive/Project - Box Office Prediction/Videos/2212.mp4	[96/221]	67	29
3	gdrive/My Drive/Project - Box Office Prediction/Videos/926.mp4	[96/275]	84	12
4	gdrive/My Drive/Project - Box Office Prediction/Videos/1939.mp4	[96/130]	62	34
5	gdrive/My Drive/Project - Box Office Prediction/Videos/637.mp4	[96/148]	53	43
6	gdrive/My Drive/Project - Box Office Prediction/Videos/2409.mp4	[96/197]	91	5
7	gdrive/My Drive/Project - Box Office Prediction/Videos/2693.mp4	[96/232]	57	39
8	gdrive/My Drive/Project - Box Office Prediction/Videos/1135.mp4	[96/216]	72	24
9	gdrive/My Drive/Project - Box Office Prediction/Videos/353.mp4	[96/176]	80	16
10	gdrive/My Drive/Project - Box Office Prediction/Videos/1549.mp4	[96/203]	85	11
11	gdrive/My Drive/Project - Box Office Prediction/Videos/2756.mp4	[96/241]	49	47
12	gdrive/My 